# The Deep Learning Project

Team members:
- Mellissa HAFIS
- Ilona LEDROGOFF

In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
!unzip project2026.zip
!ls -l

--2026-01-21 18:53:29--  https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138578548 (132M) [application/zip]
Saving to: ‘project2026.zip’

project2026.zip     100%[===================>] 132.16M  1.63MB/s    in 22s     

2026-01-21 18:53:51 (6.08 MB/s) - ‘project2026.zip’ saved [138578548/138578548]

Archive:  project2026.zip
  inflating: games.data              
  inflating: golois.cpython-312-x86_64-linux-gnu.so  
total 665408
-rw-r--r-- 1 root root 542497580 Oct  7  2022 games.data
-rwxr-xr-x 1 root root    284672 Oct  1 15:09 golois.cpython-312-x86_64-linux-gnu.so
-rw-r--r-- 1 root root 138578548 Oct  1 20:02 project2026.zip
drwxr-xr-x 1 root root      4096 Dec  9 14:42 sample_data


# Version 2

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import gc

import golois

planes = 31
moves = 361
N = 50000
epochs = 64 # 100
batch = 128
filters = 32

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

In [ ]:
policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)
policy = policy.astype ('float32')

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("Tensorflow version", tf.__version__)
print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

Tensorflow version 2.19.0
getValidation


In [ ]:
input_data.shape

(50000, 19, 19, 31)

In [ ]:
def SeparableConv2D_residual_block(x, filters):
    shortcut = x

    # First separable conv
    x = layers.SeparableConv2D(filters, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second separable conv
    x = layers.SeparableConv2D(filters, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Add skip connection
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)

    return x


# features extractor

input = keras.Input(shape=(19, 19, planes), name='board')

x = layers.Conv2D(filters, 1, padding='same', use_bias=False)(input)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

for i in range(27): # max number of blocks under the constrainte of 100k params
    x = SeparableConv2D_residual_block(x, filters)


# policy head
policy_head = layers.Conv2D(2, 1, activation='relu', padding='same')(x)
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)

# value head
value_head = layers.Conv2D(2, 1, activation='relu', padding='same')(x)
value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.001))(value_head)
value_head = layers.Dropout(0.3)(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary ()

# compilation
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), # initial value-> 0.0001
              loss={'policy': 'categorical_crossentropy', 'value': 'mse'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mae'})

# training
for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups, i * N)
    history = model.fit(input_data,
                        [policy,value],
                        epochs=1, batch_size=batch)
    if (i % 5 == 0):
        gc.collect ()
    if (i % 20 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
        model.save ('test.keras')

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ board (InputLayer)  │ (None, 19, 19,    │          0 │ -                 │
│                     │ 31)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 19, 19,    │        992 │ board[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 19, 19,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 19, 19,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 19, 19,    │      1,312 │ activation[0][0]  │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ separable_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 19, 19,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 19, 19,    │      1,312 │ activation_1[0][… │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ separable_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 19, 19,    │          0 │ add[0][0]         │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_2  │ (None, 19, 19,    │      1,312 │ activation_2[0][… │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ separable_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 19, 19,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_3  │ (None, 19, 19,    │      1,312 │ activation_3[0][… │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ separable_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 19, 19,    │          0 │ batch_normalizat

 Total params: 97,167 (379.56 KB)

 Trainable params: 93,647 (365.81 KB)

 Non-trainable params: 3,520 (13.75 KB)

epoch 1
391/391 ━━━━━━━━━━━━━━━━━━━━ 152s 189ms/step - loss: 11.5876 - policy_categorical_accuracy: 0.0035 - policy_loss: 11.2637 - value_loss: 0.2369 - value_mae: 0.3871
epoch 2
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 77ms/step - loss: 6.1749 - policy_categorical_accuracy: 0.0051 - policy_loss: 5.9640 - value_loss: 0.1312 - value_mae: 0.2992
epoch 3
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 76ms/step - loss: 6.0636 - policy_categorical_accuracy: 0.0058 - policy_loss: 5.8683 - value_loss: 0.1231 - value_mae: 0.2941
epoch 4
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 76ms/step - loss: 6.0313 - policy_categorical_accuracy: 0.0057 - policy_loss: 5.8446 - value_loss: 0.1223 - value_mae: 0.2945
epoch 5
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 77ms/step - loss: 5.9958 - policy_categorical_accuracy: 0.0071 - policy_loss: 5.8177 - value_loss: 0.1215 - value_mae: 0.2940
epoch 6
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 76ms/step - loss: 5.9472 - policy_categorical_accuracy: 0.0074 - policy_loss: 5.7771 - value_loss: 0.1212 - value_mae: 0

In [ ]:
model.save('Mellissa_HAFIS-Ilona_LEDROGOFF_SepBlocks.h5')
print("Model saved as Mellissa_HAFIS-Ilona_LEDROGOFF.h5")

Model saved as Mellissa_HAFIS-Ilona_LEDROGOFF.h5
